In [ ]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball
from skimage import color

from sklearn import preprocessing

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
from radio_beam.commonbeam import getMinVolEllipse

from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset


In [ ]:
path='data/artif_31.01/'
folders_paths=glob.glob(path+'*')
deep_folders_paths=[glob.glob(path+'/*') for path in folders_paths]
for folders_0 in deep_folders_paths:
    folder_class_name=folders_0[0].split('\\')[1]
    for folder in folders_0:
        images_paths=glob.glob(folder+'/*')
        for image_path in images_paths:
            img=io.imread(image_path)
            name=image_path.split('\\')[-1][:-4]
            # io.imsave(path+'/'+folder_class_name+'/'+name+'_'+str(time.time())+'.png',img)


In [ ]:
path='data/artif_31.01/'
folders_paths=glob.glob(path+'*')
deep_folders_paths=[glob.glob(path+'/*') for path in folders_paths]
for folders_0 in deep_folders_paths:
    folder_class_name=folders_0[0].split('\\')[1]
    for folder in folders_0:
        images_paths=glob.glob(folder+'/*')
        for image_path in images_paths:
            img=io.imread(image_path)
            name=image_path.split('\\')[-1][:-4]
            dst=path+'/'+folder_class_name+'/'+name+'_'+str(time.time())+'.jpg'
            os.rename(image_path, dst)
            # io.imsave(dst,img)


In [ ]:

def imdivide(image: np.ndarray, h: int, side: str) -> np.ndarray:
    """
    :param image: ndarray (height,width,channels)
    :param h: int scalar
    :param side: str 'left'
    :return: ndarray (height,width/2,channels)
    """
    #
    # возвращает левую или правую часть полученного изображения
    #
    height, width = image.shape
    sides = {'left': 0, 'right': 1}
    shapes = [(0, height - h, 0, width // 2), (0, height - h, width // 2, width)]
    shape = shapes[sides[side]]

    return image[shape[0]:shape[1], shape[2]:shape[3]]

In [ ]:
# old_folder_path='../datasets/original_balanced_classes'
# old_folder_path='../datasets/bc_right_sub_left_minmax_4x'
old_folder_path='../datasets/bc_left_sub_right_minmax_4x'

folders=os.listdir(old_folder_path)
# images_paths=[glob.glob(f'../data/dataset_cleaned/{folder}/*') for folder in folders]
images_paths=[glob.glob(f'{old_folder_path}/{folder}/*') for folder in folders]

new_folder_path='../datasets/bc_left_sub_right_minmax_4x_360'
# new_folder_path='../datasets/bc_right_sub_left_minmax_4x_360'

if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)

for folder in folders:
    if not os.path.exists(new_folder_path+'/'+folder):
        os.mkdir(new_folder_path+'/'+folder)

for folder_images_path in images_paths:

    folder=folder_images_path[0].split('\\')[0].split('/')[-1]
    for image_path in folder_images_path:
        image_name=image_path.split('\\')[-1]
        image=io.imread(image_path)

        left_img = imdivide(image, h=145, side='left')
        right_img = imdivide(image, h=145, side='right')
        #
        # l = 0.5
        # r = 1 - l
        # gray = np.array(left_img) * l
        #
        # gray_left = np.array(left_img)*l - np.array(right_img) * r
        # gray_right = np.array(right_img)*r-np.array(left_img) * l
        # gray_left = np.array(left_img) - np.array(right_img)
        # gray_right = np.array(right_img)-np.array(left_img)

        # img=gray_right
        # img=gray_left
        img=image

        # scaler=preprocessing.MinMaxScaler(feature_range=(0,255))
        # img=scaler.fit_transform(img).astype(np.uint8)

        width, height=image.shape
        # img1=img[0:width//2, 0:height//2]
        # img2=img[0:width//2, height//2:]
        # img3=img[width//2:, 0:height//2]
        # img4=img[width//2:, height//2:]

        img1=transform.rotate(image,90)*255
        img2=transform.rotate(image,180)*255
        img3=transform.rotate(image,270)*255

        # io.imsave(new_folder_path+'/'+folder+'/'+image_name,img)

        name=image_name.split('.')[0]

        # io.imsave(new_folder_path+'/'+folder+'/'+name+'_part_1.png',img1)
        # io.imsave(new_folder_path+'/'+folder+'/'+name+'_part_2.png',img2)
        # io.imsave(new_folder_path+'/'+folder+'/'+name+'_part_3.png',img3)
        # io.imsave(new_folder_path+'/'+folder+'/'+name+'_part_4.png',img4)


        io.imsave(new_folder_path+'/'+folder+'/'+name+'_angle_0.png',image)
        io.imsave(new_folder_path+'/'+folder+'/'+name+'_angle_90.png',img1)
        io.imsave(new_folder_path+'/'+folder+'/'+name+'_angle_180.png',img2)
        io.imsave(new_folder_path+'/'+folder+'/'+name+'_angle_270.png',img3)





In [ ]:
images=glob.glob('data/artif/mixer/*')
images=[io.imread(name) for name in images]
step=2

all_original_angles = []
for image in images:

    img = rgb2gray(image)*255
    # global_thresh = filters.threshold_otsu(img)
    global_thresh = 67
    bin_image = img > global_thresh
    bin_image=bin_image.astype(np.uint8)*255

    preproc_image=grainPreprocess.image_preprocess(bin_image)

    original_angles = grainMark.get_angles(preproc_image)

    for angle in original_angles:
        all_original_angles.append(angle)

angles, angles_set, dens_curve = grainStats.stats_preprocess(all_original_angles, step)

x = angles_set.astype(np.float64)
y = dens_curve

norm = np.sum(y)
y = y / norm

(x_gauss, y_gauss), mus, sigmas, amps = grainApprox.bimodal_gauss_approx(x, y)

# text = grainGenerate.angles_legend(len(images_list), names[i], types_dict[names[i]], step, mus, sigmas,
#                                    amps, norm)
#
# xy_gauss.append((x_gauss, y_gauss))
# xy_scatter.append((x, y))
# xy_gauss_data.append((
#     (mus[0], sigmas[0], amps[0]),
#     (mus[1], sigmas[1], amps[1])
# ))
#
# texts.append(text)

In [ ]:
cnts=grainMark.get_contours(preproc_image)
cnt_image=grainDraw.draw_edges(preproc_image[:,:,0].astype(np.float32),cnts,r=6,e_width=7,l_width=5)

fig, (ax1,ax2,ax3)=plt.subplots(1,3,figsize=(15, 15))
# fig(figsize=(10, 10))
ax1.imshow(image, cmap='gray')
ax2.imshow(preproc_image, cmap='gray')
ax3.imshow(cnt_image, cmap='gray')
# plt.savefig('mixer_preproc.png')
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(cnt_image,cmap='gray')
plt.savefig('cnt_preproc.png')
plt.show()

In [ ]:
plt.hist(angles,bins=18)
# plt.savefig('распределение углов исскуственное грубое_mixer.png')

In [ ]:
dens=np.array(dens_curve)/sum(dens_curve)
plt.figure(figsize=(10,10))
plt.plot(angles_set,dens)
plt.xlabel('angles')
plt.ylabel('probability')
plt.legend([f'Искусственные картинки mixer, шаг={step}\n Количество изображений={len(images)}'])
plt.savefig('углы_искусственные_mixer.png')
plt.show()

In [ ]:
io.imsave('atfir_preprocess.png',i)

In [ ]:
images=glob.glob('data/artif/mixer/*')
images=[io.imread(name) for name in images]
step=2

preproc_images=[]

all_original_angles = []
for image in images:

    img = rgb2gray(image)*255
    # global_thresh = filters.threshold_otsu(img)
    global_thresh = 67
    bin_image = img > global_thresh
    bin_image=bin_image.astype(np.uint8)*255

    preproc_image=grainPreprocess.image_preprocess(bin_image)
    preproc_images.append(preproc_image)

preproc_images=np.array(preproc_images)


In [ ]:
np.save('preproc_mixer.npy',preproc_images)

In [ ]:
pixel = 49.7 / 1000
all_a_beams = []
all_b_beams = []

for j, image in enumerate(preproc_images):
    a_beams, b_beams, angles, cetroids = grainMark.get_mvee_params(image, 0.2, debug=False)

    for k in range(len(a_beams)):
        all_a_beams.append(a_beams[k])
        all_b_beams.append(b_beams[k])

distances1, dist1_set, dens1_curve = grainStats.stats_preprocess(all_a_beams, step)
distances2, dist2_set, dens2_curve = grainStats.stats_preprocess(all_b_beams, step)

angles, angles_set, angles_dens_curve = grainStats.stats_preprocess(np.rad2deg(angles).astype('int32'),
                                                                    step=step)

norm1 = round(np.sum(dens1_curve), 6)
norm2 = round(np.sum(dens2_curve), 6)

x1 = np.array([dist1_set]).reshape(-1, 1) * pixel

y1 = np.log([dens1_curve / norm1]).reshape(-1, 1)

x2 = np.array([dist2_set]).reshape(-1, 1) * pixel
y2 = np.log([dens2_curve / norm2]).reshape(-1, 1)

# x1 = x1[start:end]
# y1 = y1[start:end]
#
# x2 = x2[start:end]
# y2 = y2[start:end]

(x_pred1, y_pred1), k1, b1, angle1, score1 = grainApprox.lin_regr_approx(x1, y1)
(x_pred2, y_pred2), k2, b2, angle2, score2 = grainApprox.lin_regr_approx(x2, y2)

dist_step = pixel * step



In [ ]:
# dens=np.array(dens_curve)/sum(dens_curve)
plt.figure(figsize=(7,7))
plt.plot(x_pred1, y_pred1)
plt.plot(x_pred2, y_pred2)

plt.scatter(x_pred1, y_pred1)
plt.scatter(x_pred2, y_pred2)

plt.xlabel('Length')
plt.ylabel('probability')
# plt.legend([f'Искусственные картинки mixer, шаг={step}\n Количество изображений={len(images)}'])
plt.legend(['Полуось А','Полуось B'])
plt.savefig('Полуоси_искусственные_mixer.png')
plt.show()